In [ ]:
import geopandas
import scipy.io
import numpy as np
from EchoPro import EchoPro
epro_2019 = EchoPro(init_file_path='./config_files/initialization_config.yml',
                    survey_year_file_path='./config_files/survey_year_2019_config.yml',
                    source=3,
                    bio_data_type=1,
                    age_data_status=1, 
                    exclude_age1=True)

In [ ]:
# get example data from Matlab version of EchoPro
mat = scipy.io.loadmat('../2019_consolidated_files/variogram_data.mat')

positions = mat['variogram_data'][:, :2]  # (x,y)
field = mat['variogram_data'][:, 2]   # biomass density

In [ ]:
# setup bins for semi-variogram calculation
nlag = 30 
lag_res = 0.002
bins_center = lag_res*np.arange(nlag)

bins = [bins_center[0] - lag_res/2.0]
for i in range(nlag):
    
    bins.append(bins_center[i] + lag_res/2.0)

In [ ]:
# get semi-variogram class
semi_vario = epro_2019.get_semi_variogram(positions[:, 0], 
                                          positions[:, 1], field)

In [ ]:
%%time
semi_vario.calculate_semi_variogram(bins_center)
semi_vario.gamma_standardized

In [ ]:
array([0.55957167, 0.27871057, 0.5017043 , 0.6285182 , 0.81699518,
       0.84807799, 0.85626063, 0.88805541, 0.87678417, 0.89532938,
       0.925526  , 0.92969017, 0.92621991, 0.93359212, 0.94452939,
       0.94982919, 0.92340754, 0.91565693, 0.93720898, 0.96212605,
       0.94361212, 0.93675056, 0.96621628, 0.97465567, 0.97700998,
       0.96146162, 0.98101481, 0.98020469, 0.98974528, 0.95736852])

In [ ]:
func = semi_vario.generalized_exp_bessel

In [ ]:
from scipy.optimize import curve_fit
# lag_vec, gamma
# variance, ls, exp_pow, ls_hole_eff, nugget
fit_params, pcov = curve_fit(func, bins_center, standardized_gamma, bounds=(0, [np.inf, np.inf, np.inf, 1e-14, 1e-14]))

fit_params

In [ ]:
import matplotlib.pyplot as plt

lsq_x_axis = np.linspace(np.min(bins_center), np.max(bins_center), 100)

plt.plot(bins_center, standardized_gamma, 'bo')
plt.plot(lsq_x_axis, func(lsq_x_axis, *tuple(fit_params)), 'r-')

In [ ]:
%matplotlib widget
semi_vario.view_semi_variogram()

In [ ]:
x, y, z = np.array([1, 2, 3])

In [ ]:
type(x)

In [ ]:
# import scipy.io
# import numpy as np 
# import gstools as gs 
# # get example data from Matlab version of EchoPro
# mat = scipy.io.loadmat('../2019_consolidated_files/variogram_data.mat')

# positions = mat['variogram_data'][:, :2].T  # (x,y)
# field = mat['variogram_data'][:, 2]   # biomass density

In [ ]:
# bin_center, gamma, counts = gs.vario_estimate(positions, 
#                                               field, 
#                                               bin_edges=bins, 
#                                               return_counts=True,
# #                                               direction=[np.array([0.0, np.pi/2.0])],
#                                               angles_tol=np.deg2rad(360))  #np.pi/8.0)

# print(bin_center)
# print(gamma)
# print(counts)

In [ ]:
# # fit_model = gs.Stable(dim=2)
# fit_model = gs.Exponential(dim=2)

# fit_model.fit_variogram(bin_center, gamma, nugget=True)

# ax = fit_model.plot(x_max=0.06)
# ax.scatter(bin_center, gamma)

In [ ]:
# fit_model

In [ ]:
# # enclosing box for data points
# grid_lat = np.linspace(np.min(positions[0]), np.max(positions[0]))
# grid_lon = np.linspace(np.min(positions[1]), np.max(positions[1]))

# krige = gs.krige.Ordinary(fit_model, positions, field, fit_variogram=True)
# krige.structured((grid_lat, grid_lon))

# # plot the result
# krige.plot()
# # show the fitting results
# print(krige.model)